Digital Epidemiology Dengue Transmission Model
==============================================

This code implements a comprehensive dengue transmission model for Brazilian States. The model combines:

1. A compartmental ODE system modeling dengue transmission between vectors (Aedes aegypti) 
   and humans
2. Weather-dependent parameters affecting vector biology
3. Bayesian parameter estimation using PyMC
4. Forecasting capabilities with uncertainty quantification
5. Integration with epidemiological surveillance data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import pandas as pd
import time
import requests
from numba import njit
import warnings
warnings.filterwarnings('ignore')
import pymc as pm
import pytensor.tensor as pt
from pytensor.compile.ops import as_op
from concurrent.futures import ThreadPoolExecutor, as_completed
from sympy import exp
import arviz as az
import scipy.stats as stats
import corner
import os
import pickle
import json
import mosqlient as mosq
from datetime import datetime
from geopy import Nominatim
from meteostat import Stations, Hourly

API Configuration and Model Parameters
=====================================


In [ ]:
# Dengue surveillance data API configuration
infodengue_api = "https://api.mosqlimate.org/api/datastore/infodengue/"
mosqlimate_key = {"X-UID-Key": "DavideNicola96:772df8a3-414a-438a-93b0-f326fea382e9"}
disease = "dengue"

# Primary model parameters to be estimated via Bayesian inference
par = {
    'k_v': 0.466,    # Probability of infection per bite from infected human
    'k_h': 0.314,    # Probability of infection per bite from infected vector
    'sus_per': 0.128 # Proportion of susceptible population
}

# Weather-dependent parameters for vector biology modeling
dict_weather_coeffs = {
    'A': 0.15,
    'HA': 33256.,
    'HH': 50543.,
    'TH': 301.67,
    'b0': 5.,
    'Emax': 80.,
    'Emean': 7.,
    'Evar': 2.,
    'bite': 0.00161,
}

# Fixed biological parameters based on literature
egg_lper = 0.01      # Percentage of female mosquitoes laying eggs per day
female_per = 0.5     # Proportion of female mosquitoes
mu_v = 0.02941       # Vector mortality rate (1/days)
psi_v = 0.05         # Vertical transmission rate
mu_e = 0.15          # Egg mortality rate (1/days)
alpha_v = 0.1428     # Vector incubation rate (1/days)
pi_h = 1. / 25500.   # Human birth rate (1/days)
mu_h = 1. / 25500.   # Human mortality rate (1/days)
alpha_h = 0.33       # Human incubation rate (1/days)
beta_h = 0.30        # Human recovery rate (1/days)
sigma_h = 0.0001     # Disease-induced mortality rate (1/days)

# Vectors carrying capacity
cc_v = 3

# Minimum value to prevent numerical issues
MIN_VALUE = 1e-6

# Load Brazilian geographic data for geocoding like geocode and numicipality name
ods_path = r"~/codici/RELATORIO_DTB_BRASIL_DISTRITOS.ods"
geo_data = pd.read_excel(ods_path, engine='odf')
geo_data.columns = geo_data.iloc[5].values
geo_data = geo_data.drop(index=range(0, 6))
geo_data = geo_data.reset_index(drop=True)
geo_data = geo_data[["Nome_UF","Código Município Completo", "Nome_Município"]]
geo_data.drop_duplicates(inplace=True, ignore_index=True)

Data Acquisition Functions
==========================


In [ ]:
def fetch_dengue_data_state_from_csv(state_geocodes, start_date, end_date):
    """
    Fetch dengue case data for a Brazilian state from local CSV files.
    
    This function aggregates dengue cases and population data at the state level
    for model training and validation.
    
    Parameters:
    -----------
    state_geocodes : list
        List of municipality geocodes for the target state
    start_date : str
        Start date for data extraction (YYYY-MM-DD format)
    end_date : str
        End date for data extraction (YYYY-MM-DD format)
        
    Returns:
    --------
    weekly_cases : pd.DataFrame
        Aggregated weekly dengue cases with population data
    major_cities : pd.DataFrame
        Top 10 cities by population for weather data sampling
    """
    # Load compressed dengue surveillance data
    cases_data = pd.read_csv(
        '/home/dnicola/codici/data_sprint_2025/dengue.csv.gz',
        compression='gzip',
        parse_dates=['date'],
        usecols=['date', 'geocode', 'casos', 'epiweek']
    )
    
    # Filter data for target state and date range
    cases_data = cases_data[cases_data['geocode'].isin(state_geocodes)]
    date_mask = (cases_data['date'] >= start_date) & (cases_data['date'] <= end_date)
    cases_data = cases_data[date_mask].reset_index(drop=True)

    # Aggregate cases by week (epidemiological surveillance standard)
    cases_data['date'] = pd.to_datetime(cases_data['date'])
    cases_data.set_index('date', inplace=True)
    weekly_cases = cases_data.groupby('date').agg({
        'casos': 'sum'
    }).reset_index()
    weekly_cases.columns = ['data_iniSE', 'casos']

    # Load population data for normalization
    pop_data = pd.read_csv(
        '/home/dnicola/codici/data_sprint_2025/datasus_population_2001_2024.csv.gz',
        compression='gzip'
    )

    # Filter and aggregate population data by year
    pop_data = pop_data[pop_data['geocode'].isin(state_geocodes)]
    date_mask_year = (pop_data['year'] >= pd.to_datetime(start_date).year) & (pop_data['year'] <= pd.to_datetime(end_date).year)
    pop_data = pop_data[date_mask_year].reset_index(drop=True)
    pop_data_by_year = pop_data.groupby('year').agg({
        'population': 'sum',
    }).reset_index()

    # Merge cases with population data
    weekly_cases['year'] = pd.to_datetime(weekly_cases['data_iniSE']).dt.year
    weekly_cases = weekly_cases.merge(pop_data_by_year, on='year', how='left')
    weekly_cases = weekly_cases[['data_iniSE', 'casos', 'population']]
    weekly_cases.columns = ['data_iniSE', 'casos', 'pop']
    weekly_cases = weekly_cases.sort_values('data_iniSE', ascending=True)
    weekly_cases = weekly_cases.reset_index(drop=True)
    
    # Identify major cities for weather data sampling
    latest_year = pop_data['year'].max()
    latest_pop = pop_data[pop_data['year'] == latest_year].copy()

    geo_renamed = geo_data[['Código Município Completo', 'Nome_Município']].rename(
        columns={'Código Município Completo': 'geocode'}
    )
    geo_renamed['geocode'] = geo_renamed['geocode'].astype(int)
    
    major_cities = latest_pop.merge(
        geo_renamed,
        on='geocode',
        how='left'
    )

    major_cities = major_cities[['Nome_Município', 'geocode', 'population']]
    major_cities.columns = ['city', 'geocode', 'pop']
    major_cities = major_cities.nlargest(10, 'pop').reset_index(drop=True)
    
    return weekly_cases, major_cities

def get_state_weather_data(start_date, end_date, weather_coeffs, major_cities):
    """
    Collect and average weather data from the 10 most populated cities in a state.
    
    Weather parameters are crucial for dengue transmission modeling as they
    affect vector biology (development rates, survival, biting behavior).
    
    Parameters:
    -----------
    start_date : str
        Start date for weather data
    end_date : str
        End date for weather data
    weather_coeffs : dict
        Coefficients for weather-dependent biological processes
    major_cities : pd.DataFrame
        Cities to sample weather data from
        
    Returns:
    --------
    avg_weather_data : pd.DataFrame
        State-averaged weather-dependent biological parameters
    """
    weather_data_list = []
    successful_cities = []
    
    # Collect weather data from multiple cities to reduce spatial bias
    for city in major_cities['city']:
        weather_data = weather_functions_Aedes(
            city, 'Brasil', 
            pd.to_datetime(start_date), 
            pd.to_datetime(end_date), 
            weather_coeffs
        )
        weather_data_list.append(weather_data)
        successful_cities.append(city)
    
    # Average weather-dependent parameters across cities
    avg_weather_data = weather_data_list[0].copy()
    for col in ['bite', 'egg', 'theta']:
        if col in avg_weather_data.columns:
            col_data = np.array([df[col].values for df in weather_data_list])
            avg_weather_data[col] = np.mean(col_data, axis=0)
    
    return avg_weather_data

def Briere(T, a, b, c):
    """
    Brière temperature response function for arthropod biology.
    
    This nonlinear function captures the temperature dependence of biological
    rates in ectothermic organisms like Aedes aegypti mosquitoes.
    
    Parameters:
    -----------
    T : float/array
        Temperature (°C)
    a : float
        Rate coefficient
    b : float
        Lower temperature threshold
    c : float
        Upper temperature threshold
        
    Returns:
    --------
    rate : float/array
        Temperature-dependent biological rate
    """
    T = np.clip(T, b, c)
    return a * T * (T - b) * np.sqrt(c - T)

def get_location(city_name, country_name):
    """
    Geocode city location for weather station selection.
    
    Uses Nominatim geocoding service with rate limiting to avoid
    service overload.
    """
    time.sleep(3)  # Rate limiting

    geolocator = Nominatim(user_agent="weather_app", timeout=3)
    location = geolocator.geocode(f"{city_name}, {country_name}")
    
    return location

def get_altitude(latitude, longitude):
    """
    Get elevation data for evaporation calculations.
    
    Altitude affects temperature and humidity patterns, which influence
    mosquito breeding site availability.
    """
    elevation_url = "https://api.opentopodata.org/v1/test-dataset"
    elevation_params = {
        "locations": f"{latitude},{longitude}"
    }
    elevation_response = requests.get(elevation_url, params=elevation_params).json()
    altitude = elevation_response['results'][0]['elevation']

    return altitude

def get_nearest_station(city_name, country_name, end_date):
    """
    Find nearest meteorological station with data coverage.
    
    Prioritizes stations with complete temporal coverage for the
    analysis period.
    """
    end_date = end_date.strftime("%Y-%m-%d")
    location = get_location(city_name, country_name)

    if location:
        stations = Stations()
        nearest_station = stations.nearby(location.latitude, location.longitude).fetch()
        nearest_station.dropna(subset=["hourly_end"], inplace=True)
        nearest_station = nearest_station[(nearest_station["hourly_end"] >= end_date) & (nearest_station["hourly_start"] <= end_date)]
        return nearest_station
    else:
        raise ValueError(f"Coordinates not found for {city_name}, {country_name}")

def fetch_weather_data(city_name, country_name, start_date, end_date):
    """
    Download meteorological data from nearest weather station.
    
    Retrieves daily temperature, humidity, precipitation, and wind data
    needed for vector biology modeling.
    """
    try:
        i = 0
        while True:
            nearest_station = get_nearest_station(city_name, country_name, end_date).iloc[[i]]
            # Resample hourly data to daily means and fill missing values
            data = Hourly(nearest_station, start=start_date, end=end_date).normalize().fetch().resample('D').mean().fillna(0)
            data = data[['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd']]
            if not data.empty:
                return data
            else:
                i += 1
    except ValueError as e:
        print(f"Error finding nearest station for {city_name}, {country_name}: {e}")
    except Exception as e:
        print(f"Error fetching weather data: {e}")
    return None

def weather_functions_Aedes(city_name, country_name, start_date, end_date, coeffs):
    """
    Calculate weather-dependent Aedes aegypti biological parameters.
    
    This function transforms meteorological data into biologically meaningful
    parameters for the dengue transmission model:
    - theta: temperature-dependent development rate
    - bite: temperature-dependent biting rate  
    - egg: moisture-dependent egg laying rate
    
    Parameters:
    -----------
    city_name : str
        Target city name
    country_name : str
        Target country
    start_date : datetime
        Start date for weather data
    end_date : datetime
        End date for weather data
    coeffs : dict
        Weather response coefficients
        
    Returns:
    --------
    weather_data : pd.DataFrame
        Daily weather data with calculated biological parameters
    """
    K = 273.15  # Kelvin conversion constant

    A, HA, HH, TH, b0, Emax, Emean, Evar, bite = coeffs.values()
    location = get_location(city_name, country_name)
    latitude, longitude = location.latitude, location.longitude
    altitude = get_altitude(latitude, longitude)

    weather_data = fetch_weather_data(city_name, country_name, start_date, end_date)
    # Smooth temperature with 7-day rolling mean to reduce noise
    weather_data['temp_r'] = weather_data['temp'].rolling(7, min_periods=1).mean()

    # Initialize biological parameter arrays
    Theta = np.zeros(shape = weather_data.shape[0], dtype = np.float64)
    Egg = np.zeros(shape = weather_data.shape[0], dtype = np.float64)
    Bite = np.zeros(shape = weather_data.shape[0], dtype = np.float64)
    
    for t in range(weather_data.shape[0]):
        Temp = weather_data['temp_r'][t]
    
        # Calculation of development rate
        Theta[t] = A * ((Temp + K) / 298.15) * exp((HA / (1.987)) * (1 / 298.15 - 1 / (Temp + K))) * \
            (1 + exp((HH / (1.987)) * (1 / TH - 1 / (Temp + K))))
            
        # Brière function for biting rate
        Bite[t] = Briere(Temp, bite, 13.35, 40.08)

        # Moisture-dependent egg laying (requires 3-day accumulation)
        if t >= 3:
            Precipitation = 0
            Evaporation = 0
            # Calculate 3-day moisture balance
            for d in range(t - 2, t + 1):
                temp = weather_data['temp'][d]
                Td = weather_data['dwpt'][d]

                Precipitation += weather_data.iloc[d,3]
                Evaporation += (700 * (temp + 0.006 * altitude)) / ((100 - latitude) * (80 - temp)) + 15 * (temp - Td) / (80 - temp)

            Moisture = Precipitation - Evaporation
            # Sigmoid response to moisture availability
            Egg[t] = b0 + Emax / (1 + exp(-(Moisture - Emean) / Evar))
        
    weather_data['theta'] = Theta
    weather_data['egg'] = Egg
    weather_data['bite'] = Bite
    
    # Remove first 3 days (incomplete moisture calculations)
    weather_data = weather_data.iloc[3:,:]
    
    return weather_data


Dengue Transmission Model (Compartmental ODE System)
===================================================


In [ ]:
@njit(fastmath=True, cache=True)
def ode_system(y, k_v, k_h, pi_v, theta_v, b):
    """
    Dengue transmission ODE system with vector and human compartments.
    
    This function implements a SEIR-type model for both vectors and humans:
    
    Vector compartments:
    - Pv: Pre-emergence (pupae/larvae)
    - Sv: Susceptible vectors  
    - Ev: Exposed vectors (infected but not infectious)
    - Qv: Pre-emergence from infected vectors (vertical transmission)
    - Iv: Infectious vectors
    - Dv: Dead vectors (cumulative)
    
    Human compartments:
    - Sh: Susceptible humans
    - Eh: Exposed humans (infected but not infectious) 
    - Ih: Infectious humans
    - Rh: Recovered humans
    - Dh: Dead humans (cumulative)
    
    Parameters:
    -----------
    y : array
        Current state vector [Pv, Sv, Ev, Qv, Iv, Dv, Sh, Eh, Ih, Rh, Dh]
    k_v : float
        Vector competence (transmission probability human→vector)
    k_h : float  
        Human susceptibility (transmission probability vector→human)
    pi_v : float
        Egg laying rate (temperature-dependent)
    theta_v : float
        Development rate (temperature-dependent) 
    b : float
        Biting rate (temperature-dependent)
        
    Returns:
    --------
    dydt : array
        Time derivatives for each compartment
    """
    Pv, Sv, Ev, Qv, Iv, Dv, Sh, Eh, Ih, Rh, Dh = y
    
    # Calculate population sizes
    Nv = Sv + Ev + Iv  # Total vector population
    Nh = Sh + Eh + Ih + Rh  # Total human population
    
    # Density-dependent mortality (competition for resources)
    mortality_factor = mu_v * (Nv / (Nh * cc_v))
    
    # Force of infection (transmission rates)
    infection_rate_v = b * k_v * (Ih / Nh)  # Human to vector
    infection_rate_h = b * k_h * (Iv / Nh)  # Vector to human
    
    # Vector compartment dynamics
    dpv = egg_lper * pi_v * (Nv - psi_v * Iv) - (mu_e + female_per * theta_v) * Pv
    dsv = female_per * theta_v * Pv - (mortality_factor + infection_rate_v) * Sv
    dev = infection_rate_v * Sv - (mortality_factor + alpha_v) * Ev
    dqv = egg_lper * pi_v * psi_v * Iv - (mu_e + female_per * theta_v) * Qv  # Vertical transmission
    div = female_per * theta_v * Qv + alpha_v * Ev - mortality_factor * Iv
    ddv = mortality_factor * Iv
    
    # Human compartment dynamics
    dsh = pi_h * Nh - (mu_h + infection_rate_h) * Sh
    deh = infection_rate_h * Sh - (mu_h + alpha_h) * Eh
    dih = alpha_h * Eh - (mu_h + beta_h + sigma_h) * Ih
    drh = beta_h * Ih - mu_h * Rh
    ddh = (mu_h + sigma_h) * Ih
    
    return np.array([dpv, dsv, dev, dqv, div, ddv, dsh, deh, dih, drh, ddh])

@njit(fastmath=True, cache=True)
def rk4_step(y, dt, k_v, k_h, pi_v, theta_v, b):
    """
    Fourth-order Runge-Kutta integration step.
    
    Provides numerical integration of the ODE system with improved
    accuracy compared to Euler's method.
    """
    k1 = ode_system(y, k_v, k_h, pi_v, theta_v, b)
    k2 = ode_system(y + 0.5 * dt * k1, k_v, k_h, pi_v, theta_v, b)
    k3 = ode_system(y + 0.5 * dt * k2, k_v, k_h, pi_v, theta_v, b)
    k4 = ode_system(y + dt * k3, k_v, k_h, pi_v, theta_v, b)
    
    return y + (dt / 6.0) * (k1 + 2 * k2 + 2 * k3 + k4)

@njit(fastmath=True, cache=True)
def initial_conditions_fast(tot_cases, tot_pop, sus_per):
    """
    Calculate epidemiologically consistent initial conditions.
    
    Estimates initial compartment sizes based on observed case data
    and epidemiological parameters, assuming quasi-equilibrium.
    
    Parameters:
    -----------
    tot_cases : float
        Total observed cases in first week
    tot_pop : float
        Total population size
    sus_per : float
        Proportion of susceptible population
        
    Returns:
    --------
    y0 : array
        Initial state vector for ODE integration
    """
    # Estimate initial infectious humans from case data
    i0 = max(int((tot_cases / (7 * beta_h)) + 0.5), 1) #Infectious humans
    e0 = int((i0 / alpha_h) + 0.5)  # Exposed humans
    s0 = int(sus_per * tot_pop  + 0.5)  # Susceptible humans
    r0 = int(tot_pop - (i0 + e0 + s0))  # Recovered humans (remainder)

    # Calculate vector populations proportional to human infections
    ev0 = int(cc_v * e0 + 0.5)  # Exposed vectors
    iv0 = int(cc_v * i0 + 0.5)  # Infectious vectors
    sv0 = int(cc_v * (s0 + r0) + 0.5)  # Susceptible vectors
    qv0 = int(cc_v * (i0 + e0) + 0.5)  # Pre-emergence (vertical transmission)
    pv0 = int(cc_v * (s0 + r0) + 0.5)  # Pre-emergence (normal)
    
    # Final human compartments
    sh0 = int(tot_pop - (i0 + e0 + r0))
    eh0 = e0
    ih0 = i0
    rh0 = r0
    
    return np.array([float(pv0), float(sv0), float(ev0), float(qv0), float(iv0), 0.0, 
                     float(sh0), float(eh0), float(ih0), float(rh0), 0.0])

@njit(fastmath=True, cache=True)
def simulate_dengue_fast(k_v, k_h, sus_per, tot_cases, tot_pop, 
                        egg_lrate, egg_drate, bite_rate, days):
    """
    Fast simulation of dengue transmission dynamics.
    
    Integrates the ODE system over the specified time period with
    time-varying weather-dependent parameters.
    
    Parameters:
    -----------
    k_v, k_h, sus_per : float
        Model parameters to be estimated
    tot_cases, tot_pop : float
        Initial epidemic conditions
    egg_lrate, egg_drate, bite_rate : array
        Time-varying weather-dependent parameters
    days : int
        Simulation duration
        
    Returns:
    --------
    results : array
        Time series of all compartments [days x 11]
    """
    # Initialize with epidemiologically consistent conditions
    y = initial_conditions_fast(tot_cases, tot_pop, sus_per)
    
    results = np.zeros((days, 11))
    results[0] = y
    
    dt = 1.0  # Daily time step
    
    # Integrate ODE system day by day
    for i in range(1, days):
        # Use weather parameters for current day (with bounds checking)
        idx = min(i, len(egg_lrate) - 1)
        pi_v = egg_lrate[idx]
        theta_v = egg_drate[idx]
        b = bite_rate[idx]
        
        # Runge-Kutta integration step
        y = rk4_step(y, dt, k_v, k_h, pi_v, theta_v, b)        
        # Prevent negative populations
        y = np.maximum(y, MIN_VALUE)
        
        results[i] = y
    
    return results

@njit(fastmath=True, cache=True)
def calculate_weekly_cases_fast(results):
    """
    Calculate weekly case incidence from compartment dynamics.
    
    Computes new cases by tracking transitions from compartments,
    aggregated to weekly totals for comparison with surveillance data.
    
    Parameters:
    -----------
    results : array
        Daily time series from ODE simulation
        
    Returns:
    --------
    weekly_cases : array
        Weekly case counts
    """
    Ih = results[:, 8]  # Infectious humans
    Rh = results[:, 9]  # Recovered humans  
    Dh = results[:, 10] # Dead humans
    
    days = len(results)
    new_cases = np.zeros(days)
    
    # Calculate daily incidence as net flow out of infectious compartment
    new_cases[0] = Ih[0]
    for i in range(1, days):
        new_Ih = max(0.0, Ih[i] - Ih[i-1])  # New infections
        new_Rh = max(0.0, Rh[i] - Rh[i-1])  # New recoveries
        new_Dh = max(0.0, Dh[i] - Dh[i-1])  # New deaths
        new_cases[i] = new_Ih + new_Rh + new_Dh
    
    # Aggregate to weekly totals
    weeks = days // 7
    weekly_cases = np.zeros(weeks)
    
    for w in range(weeks):
        start_idx = w * 7
        end_idx = min(start_idx + 7, days)
        weekly_cases[w] = np.sum(new_cases[start_idx:end_idx])
    
    return weekly_cases

def simulate_dengue_wrapper(params, egg_lrate, egg_drate, bite_rate, cases_df, days):
    """
    Wrapper function for dengue simulation with data formatting.
    
    Provides interface between parameter estimation and core simulation,
    handling data type conversions and result formatting.
    """
    k_v = params['k_v']
    k_h = params['k_h']
    sus_per = params['sus_per']
    
    tot_cases = float(cases_df['casos'].iloc[0])
    tot_pop = float(cases_df['pop'].iloc[0])
    
    # Run core simulation
    results = simulate_dengue_fast(k_v, k_h, sus_per, tot_cases, tot_pop,
                                  egg_lrate, egg_drate, bite_rate, days)
    
    # Calculate weekly case incidence
    weekly_cases = calculate_weekly_cases_fast(results)
    
    # Extract population sizes for R0 calculation
    Nv = results[-1, 1] + results[-1, 2] + results[-1, 4]  # Vector population
    Nh = results[-1, 6] + results[-1, 7] + results[-1, 8] + results[-1, 9]  # Human population
    
    return weekly_cases, np.full(days, Nv), np.full(days, Nh)

@njit(fastmath=True, cache=True)
def Basic_Reproduction_Number_fast(k_h, k_v, mean_bite, Nv, Nh):
    """
    Calculate basic reproduction number (R0) for dengue transmission.
    
    R0 represents the expected number of secondary infections produced by
    one infected individual in a completely susceptible population.
    
    For vector-borne diseases, R0 includes both human-vector and vector-human
    transmission cycles.
    
    Parameters:
    -----------
    k_h, k_v : float
        Transmission probabilities
    mean_bite : float
        Average biting rate
    Nv, Nh : float
        Vector and human population sizes
        
    Returns:
    --------
    R0 : float
        Basic reproduction number
    """
    if Nh < 1:
        Nh = 1
    if Nv < 1:
        Nv = 1
    
    # Vector infection probability and duration
    term1 = (mean_bite * k_v * Nv) / (Nh * (alpha_v + mu_v * Nv / (Nh * cc_v)))
    # Human infection probability
    term2 = (mean_bite * k_h) / (alpha_h + mu_h)
    # Vector infectious duration
    term3 = alpha_v / (mu_v * Nv / (Nh * cc_v))
    # Human infectious duration
    term4 = alpha_h / (beta_h + mu_h + sigma_h)
    
    # R0 is geometric mean of transmission cycle components
    R0 = np.sqrt(term1 * term2 * term3 * term4)
    
    return R0


Model Persistence and Results Management
=======================================

In [ ]:
def save_posterior_results(trace, state_cases_df, weather_data_df, city, start_date, end_date, 
                          best_fit_params, R0_best_fit, save_dir="./saved_models/"):
    """
    Save Bayesian posterior samples and metadata for future forecasting.
    
    Preserves all information needed to generate forecasts without re-fitting
    the model, including parameter samples, training data, and model configuration.
    
    Parameters:
    -----------
    trace : arviz.InferenceData
        PyMC posterior samples from MCMC
    state_cases_df : pd.DataFrame
        Training case data
    weather_data_df : pd.DataFrame
        Training weather data
    city : str
        State/region name
    start_date, end_date : str
        Training period dates
    best_fit_params : dict
        Maximum a posteriori parameter estimates
    R0_best_fit : float
        R0 calculated with best-fit parameters
    save_dir : str
        Directory for saved model files
        
    Returns:
    --------
    timestamp : str
        Unique identifier for saved model
    """
    os.makedirs(save_dir, exist_ok=True)
    
    # Extract posterior parameter samples for forecasting
    posterior_data = {
        'k_v_samples': trace.posterior.k_v.values.flatten(),
        'k_h_samples': trace.posterior.k_h.values.flatten(),
        'sus_per_samples': trace.posterior.sus_per.values.flatten(),
        'n_samples': len(trace.posterior.k_v.values.flatten())
    }
    
    # Store model metadata and configuration
    metadata = {
        'city': city,
        'training_start_date': start_date,
        'training_end_date': end_date,
        'best_fit_params': best_fit_params,
        'R0_best_fit': R0_best_fit,
        'training_data_shape': state_cases_df.shape,
        'weather_coeffs': dict_weather_coeffs,
        'model_parameters': {
            'egg_lper': egg_lper,
            'female_per': female_per,
            'mu_v': mu_v,
            'psi_v': psi_v,
            'mu_e': mu_e,
            'alpha_v': alpha_v,
            'pi_h': pi_h,
            'mu_h': mu_h,
            'alpha_h': alpha_h,
            'beta_h': beta_h,
            'sigma_h': sigma_h,
            'cc_v': cc_v
        }
    }
    
    # Store training data for reference
    training_data = {
        'state_cases_df': state_cases_df.to_dict(),
        'weather_data_df': weather_data_df.to_dict()
    }
    
    # Create unique timestamp identifier
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save files with pickle for complex objects, JSON for metadata
    with open(f"{save_dir}posterior_samples_{city}_{timestamp}.pkl", 'wb') as f:
        pickle.dump(posterior_data, f)
    
    with open(f"{save_dir}metadata_{city}_{timestamp}.json", 'w') as f:
        json.dump(metadata, f, indent=2, default=str)
    
    with open(f"{save_dir}training_data_{city}_{timestamp}.pkl", 'wb') as f:
        pickle.dump(training_data, f)
    
    return timestamp

def load_posterior_results(timestamp, city, save_dir="./saved_models/"):
    """
    Load previously saved model results for forecasting.
    
    Reconstructs all necessary components for generating forecasts from
    a previously trained model.
    
    Parameters:
    -----------
    timestamp : str
        Model identifier from training
    city : str
        State/region name
    save_dir : str
        Directory containing saved models
        
    Returns:
    --------
    posterior_data : dict
        Parameter samples from posterior distribution
    metadata : dict
        Model configuration and training information
    training_data : dict
        Original training datasets
    """
    with open(f"{save_dir}posterior_samples_{city}_{timestamp}.pkl", 'rb') as f:
        posterior_data = pickle.load(f)
    
    with open(f"{save_dir}metadata_{city}_{timestamp}.json", 'r') as f:
        metadata = json.load(f)
    
    with open(f"{save_dir}training_data_{city}_{timestamp}.pkl", 'rb') as f:
        training_data = pickle.load(f)
    
    return posterior_data, metadata, training_data


Model Training and Parameter Estimation
=======================================

In [ ]:
def fit_function(state, start_date, end_date, progress_bar_bool=True):
    """
    Fit dengue transmission model using Bayesian parameter estimation.
    
    This function performs the complete model training pipeline:
    1. Data collection and preprocessing
    2. Weather parameter calculation
    3. Bayesian inference using MCMC
    4. Result validation and storage
    
    Parameters:
    -----------
    state : str
        Brazilian state name for analysis
    start_date : str
        Training period start date (YYYY-MM-DD)
    end_date : str
        Training period end date (YYYY-MM-DD)
    progress_bar_bool : bool
        Show MCMC progress bar
        
    Returns:
    --------
    timestamp : str
        Unique identifier for saved model
    best_fit_params : dict
        Maximum a posteriori parameter estimates
    R0_best_fit : float
        Basic reproduction number with best-fit parameters
    """
    # Extend weather data collection to account for initial conditions
    weather_start_date = str(datetime.date(datetime.strptime(start_date, "%Y-%m-%d") - timedelta(days=2)))
    date_difference = datetime.strptime(end_date, "%Y-%m-%d") - datetime.strptime(start_date, "%Y-%m-%d")
    days = date_difference.days

    # Ensure simulation period aligns with weekly surveillance data
    if days % 7 != 0:
        end_date = str(datetime.date(datetime.strptime(end_date, "%Y-%m-%d") - timedelta(days=days % 7) + timedelta(days=7)))
        date_difference = datetime.strptime(end_date, "%Y-%m-%d") - datetime.strptime(start_date, "%Y-%m-%d")
        days = date_difference.days

    # Get state-specific municipality codes for data aggregation
    geo_data_state = geo_data[geo_data['Nome_UF'] == state]
    state_geocodes = geo_data_state['Código Município Completo'].astype(int).tolist()

    # Collect epidemiological surveillance data
    csv_state_cases_df, major_cities = fetch_dengue_data_state_from_csv(
        state_geocodes,
        pd.to_datetime(start_date),
        pd.to_datetime(end_date)
    )

    # Collect and process weather data for vector biology parameters
    weather_data_df = get_state_weather_data(
        weather_start_date,
        end_date,
        dict_weather_coeffs,
        major_cities
    )

    # Extract time-varying biological parameters for model
    bite_rate = np.array(weather_data_df['bite'], dtype=np.float64)
    egg_laying_rate = np.array(weather_data_df['egg'], dtype=np.float64)
    egg_development_rate = np.array(weather_data_df['theta'], dtype=np.float64)
    observed_cases = csv_state_cases_df['casos'].values

    # Define PyMC model interface for Bayesian inference
    @as_op(itypes=[pt.dscalar, pt.dscalar, pt.dscalar], otypes=[pt.dvector])
    def dengue_sim_fast(k_v, k_h, sus_per):
        """
        PyMC interface to dengue simulation model.
        
        Converts PyMC tensor operations to numpy for efficient computation
        while maintaining gradient information for MCMC sampling.
        """
        params = {"k_v": float(k_v), "k_h": float(k_h), "sus_per": float(sus_per)}
        try:
            weekly_cases, _, _ = simulate_dengue_wrapper(params, egg_laying_rate, egg_development_rate,
                                                         bite_rate, csv_state_cases_df, days)
            return weekly_cases[:len(csv_state_cases_df)]
        except:
            # Return small positive values if simulation fails
            return np.full(len(csv_state_cases_df), 1e-6)

    # Define Bayesian model with informative priors
    with pm.Model() as dengue_model:
        # Prior distributions based on biological knowledge and literature
        k_v = pm.Beta("k_v", alpha=2, beta=20)      # Vector competence (low probability)
        k_h = pm.Beta("k_h", alpha=8, beta=5.5)     # Human susceptibility (moderate)
        sus_per = pm.Beta("sus_per", alpha=3, beta=20)  # Susceptible proportion (low)

        # Link model predictions to observations
        cases_mean = dengue_sim_fast(k_v, k_h, sus_per)
        cases_safe = pm.math.maximum(cases_mean, 1e-6)  # Prevent zeros
        
        # Poisson likelihood for case counts (appropriate for count data)
        Y_obs = pm.Poisson("Y_obs", mu=cases_safe, observed=observed_cases)

        # Use differential evolution MCMC for complex posterior geometry
        step = pm.DEMetropolisZ()
        trace = pm.sample(draws=5000, tune=5000, chains=6, step=step,
                          random_seed=42, discard_tuned_samples=True, 
                          cores=6, progressbar=progress_bar_bool)

    # Extract best-fit parameters (posterior medians for robustness)
    k_v_median = np.median(trace.posterior.k_v.values)
    k_h_median = np.median(trace.posterior.k_h.values)
    sus_per_median = np.median(trace.posterior.sus_per.values)

    best_fit_params = {'k_v': k_v_median, 'k_h': k_h_median, 'sus_per': sus_per_median}
    
    # Generate best-fit trajectory for model validation
    weekly_best_fit, Nv_best, Nh_best = simulate_dengue_wrapper(
        best_fit_params, egg_laying_rate, egg_development_rate, bite_rate, csv_state_cases_df, days
    )

    # Calculate epidemiological indicator (R0)
    mean_bite = np.mean(bite_rate)
    R0_best_fit = Basic_Reproduction_Number_fast(k_h_median, k_v_median, mean_bite, Nv_best[0], Nh_best[0])

    # Save complete model results for future use
    timestamp = save_posterior_results(
        trace, csv_state_cases_df, weather_data_df, state, start_date, end_date,
        best_fit_params, R0_best_fit
    )
    
    return timestamp, best_fit_params, R0_best_fit


Forecasting and Prediction
==========================

In [ ]:
def forcast_function(state, forecast_start_date, forecast_end_date, metadata, posterior_data):
    """
    Generate probabilistic dengue forecasts using trained model.
    
    Uses posterior parameter samples to generate ensemble forecasts with
    full uncertainty quantification. Handles weather data extrapolation
    for forecasts beyond available meteorological data.
    
    Parameters:
    -----------
    state : str
        Target state for forecasting
    forecast_start_date : str
        Forecast period start (YYYY-MM-DD)
    forecast_end_date : str
        Forecast period end (YYYY-MM-DD)
    metadata : dict
        Model configuration from training
    posterior_data : dict
        Parameter samples from Bayesian fitting
        
    Returns:
    --------
    forecast_df : pd.DataFrame
        Probabilistic forecasts with confidence intervals
    control_cases : pd.DataFrame
        Observed data for forecast validation
    """
    time.sleep(1)  # Rate limiting for API calls

    # Parse forecast period
    forecast_start = datetime.strptime(forecast_start_date, "%Y-%m-%d")
    forecast_end = datetime.strptime(forecast_end_date, "%Y-%m-%d")
    weather_start = forecast_start - timedelta(days=2)

    date_difference = forecast_end - forecast_start
    forecast_days = date_difference.days

    # Align forecast period with weekly reporting
    if forecast_days % 7 != 0:
        forecast_end = forecast_end - timedelta(days=forecast_days % 7) + timedelta(days=7)
        forecast_days = (forecast_end - forecast_start).days
    
    # Get recent case data for initial conditions
    previous_end = forecast_start - timedelta(days=1)
    previous_start = previous_end - timedelta(days=30)

    # Collect geographic and population data
    geo_data_state = geo_data[geo_data['Nome_UF'] == state]
    state_geocodes = geo_data_state['Código Município Completo'].astype(int).tolist()
    
    # Get initial conditions from recent surveillance data
    previous_year_cases, _ = fetch_dengue_data_state_from_csv(
        state_geocodes,
        pd.to_datetime(previous_start),
        pd.to_datetime(previous_end)
    )
    
    # Get validation data (observed cases during forecast period)
    control_cases, major_cities = fetch_dengue_data_state_from_csv(
        state_geocodes,
        pd.to_datetime(forecast_start_date),
        pd.to_datetime(forecast_end_date)
    )
    
    # Extract initial epidemic conditions
    last_week_cases = previous_year_cases.iloc[-1]['casos']
    population = previous_year_cases.iloc[-1]['pop']
    
    # Get weather parameters from training metadata
    weather_coeffs = metadata['weather_coeffs']
    
    # Handle weather data availability (limited by meteorological services)
    cutoff_date = datetime.strptime("2025-06-09", "%Y-%m-%d")
    if forecast_end <= cutoff_date:
        # Use real weather data when available
        weather_data_forecast = get_state_weather_data(
            pd.to_datetime(weather_start.strftime("%Y-%m-%d")), 
            pd.to_datetime(forecast_end.strftime("%Y-%m-%d")), 
            weather_coeffs, 
            major_cities
        )
    else:
        # Extrapolate weather data beyond availability
        weather_data_forecast = get_state_weather_data(
            pd.to_datetime(weather_start.strftime("%Y-%m-%d")), 
            pd.to_datetime(cutoff_date.strftime("%Y-%m-%d")), 
            weather_coeffs, 
            major_cities
        )
        
        # Prepare for temporal extrapolation
        weather_data_forecast = weather_data_forecast.rename_axis('time').reset_index()
        weather_data_forecast['time'] = pd.to_datetime(weather_data_forecast['time'])
        weather_data_forecast = weather_data_forecast.set_index('time')
        full_dates = pd.date_range(start=weather_data_forecast.index.min(), end=forecast_end, freq='D')
        weather_data_forecast = weather_data_forecast.reindex(full_dates)
        # Linear interpolation for missing weather data
        weather_data_forecast = weather_data_forecast.interpolate(method='linear', limit_direction='both')
        weather_data_forecast = weather_data_forecast.reset_index().rename(columns={'index': 'date'})

    # Extract weather-dependent biological parameters
    bite_rate_forecast = np.array(weather_data_forecast['bite'], dtype=np.float64)
    egg_laying_rate_forecast = np.array(weather_data_forecast['egg'], dtype=np.float64)
    egg_development_rate_forecast = np.array(weather_data_forecast['theta'], dtype=np.float64)
    
    # Prepare initial conditions for forecast simulation
    initial_conditions_df = pd.DataFrame({
        'casos': [last_week_cases],
        'pop': [population]
    })

    # Generate ensemble forecasts using posterior samples
    n_total_samples = len(posterior_data['k_v_samples'])
    forecast_results = []
    
    # Run forecast simulation for each posterior sample
    for i, idx in enumerate(range(n_total_samples)):
        sample_params = {
            'k_v': posterior_data['k_v_samples'][idx],
            'k_h': posterior_data['k_h_samples'][idx],
            'sus_per': posterior_data['sus_per_samples'][idx]
        } 
        
        # Generate forecast trajectory with current parameter sample
        weekly_forecast, Nv_forecast, Nh_forecast = simulate_dengue_wrapper(
            sample_params, egg_laying_rate_forecast, egg_development_rate_forecast,
            bite_rate_forecast, initial_conditions_df, forecast_days
        )
        
        forecast_results.append(weekly_forecast)

    # Convert to array for statistical analysis
    forecast_results = np.array(forecast_results)
    
    # Calculate prediction intervals at multiple confidence levels
    confidence_levels = [50, 80, 90, 95]
    forecast_stats = {}
    for level in confidence_levels:
        lower_bound = 50 - level / 2
        upper_bound = 50 + level / 2
        forecast_stats[f'ci_{level}_lower'] = np.percentile(forecast_results, lower_bound, axis=0)
        forecast_stats[f'ci_{level}_upper'] = np.percentile(forecast_results, upper_bound, axis=0)

    # Calculate central tendency and dispersion
    forecast_stats['median'] = np.median(forecast_results, axis=0)
    forecast_stats['std'] = np.std(forecast_results, axis=0)

    # Format forecast results
    forecast_df = pd.DataFrame({
        'week': range(len(forecast_stats['median'])),
        'median': forecast_stats['median'],
        'std': forecast_stats['std']
    })
    
    # Add confidence intervals
    for level in confidence_levels:
        forecast_df[f'ci_{level}_lower'] = forecast_stats[f'ci_{level}_lower']
        forecast_df[f'ci_{level}_upper'] = forecast_stats[f'ci_{level}_upper']

    # Add temporal dimension
    forecast_dates = pd.date_range(start=forecast_start, freq='W', periods=len(forecast_stats['median']))
    forecast_df['date'] = forecast_dates

    return forecast_df, control_cases


Results Submission and Competition Interface
===========================================

In [ ]:
def upload_forecast(state, forecast_df, adm_1_map,
                    model_id=134,
                    description_template="2024-25 Dengue Forecast for {}",
                    commit_hash='df1c695eaa5af5edb125a9d4ec72a4d2528c7411',
                    api_key='DavideNicola:a4ca210d-7d0e-45c9-8cab-1237127d22af'):
    """
    Upload forecast results to digital epidemiology competition platform.
    
    Formats probabilistic forecasts according to competition requirements
    and submits via API for evaluation against held-out surveillance data.
    
    Parameters:
    -----------
    state : str
        Brazilian state code
    forecast_df : pd.DataFrame
        Probabilistic forecast results
    adm_1_map : dict
        State name to code mapping
    model_id : int
        Competition model identifier
    description_template : str
        Forecast description template
    commit_hash : str
        Model version identifier
    api_key : str
        Competition platform API key
    """
    predict_date = datetime.now().strftime('%Y-%m-%d')

    # Rename columns to match competition format requirements
    df_renamed = forecast_df.rename(columns={
        'ci_95_lower': 'lower_95',
        'ci_90_lower': 'lower_90',
        'ci_80_lower': 'lower_80',
        'ci_50_lower': 'lower_50',
        'median': 'pred',
        'ci_50_upper': 'upper_50',
        'ci_80_upper': 'upper_80',
        'ci_90_upper': 'upper_90',
        'ci_95_upper': 'upper_95'
    })

    # Ensure all required columns are present
    required_columns = ['lower_95', 'lower_90', 'lower_80', 'lower_50',
                        'pred', 'upper_50', 'upper_80', 'upper_90', 'upper_95', 'date']

    df_final = df_renamed[required_columns].copy()
    adm_1_code = adm_1_map[state]

    # Submit forecast to competition platform
    mosq.upload_prediction(
        model_id=model_id,
        description=description_template.format(state),
        commit=commit_hash,
        predict_date=predict_date,
        prediction=df_final,
        adm_1=adm_1_code,
        api_key=api_key
    )


Main Execution Pipeline
======================

In [ ]:
# State administrative mapping for competition submission
adm_1_map = {
    'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA', 'Ceará': 'CE',
    'Distrito Federal': 'DF', 'Goiás': 'GO', 'Maranhão': 'MA', 'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS', 'Minas Gerais': 'MG', 'Pará': 'PA', 'Paraíba': 'PB',
    'Paraná': 'PR', 'Pernambuco': 'PE', 'Piauí': 'PI', 'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS', 'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC',
    'São Paulo': 'SP', 'Sergipe': 'SE', 'Tocantins': 'TO'
}

# Complete list of Brazilian states for analysis ()
brazilian_states = [
    'Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', 'Ceará',
    'Distrito Federal', 'Goiás', 'Maranhão', 'Mato Grosso',
    'Mato Grosso do Sul', 'Minas Gerais', 'Pará', 'Paraíba',
    'Paraná', 'Pernambuco', 'Piauí', 'Rio de Janeiro', 'Rio Grande do Norte',
    'Rio Grande do Sul', 'Rondônia', 'Roraima', 'Santa Catarina',
    'São Paulo', 'Sergipe', 'Tocantins'
]

Define temporal periods for model training and forecasting evaluation.

In [ ]:
# Training period: Historical data for parameter estimation
train_start_date = "2020-10-10"  # Start of training period
train_end_date = "2023-06-30"    # End of training period

# Forecast period: Out-of-sample prediction evaluation
forecast_start_date = "2023-10-08"  # Start of forecast period
forecast_end_date = "2024-10-06"    # End of forecast period

# Results storage
fit_results_by_state = {}      # Training results for each state
forecast_results_by_state = {} # Forecast results for each state
upload = True                  # Submit to competition platform


Execute complete modeling pipeline for all Brazilian states.

This loop performs:
1. Bayesian parameter estimation using historical data
2. Probabilistic forecasting with uncertainty quantification  
3. Competitive submission to digital epidemiology platform


In [ ]:
for state in brazilian_states:
    # === MODEL TRAINING PHASE ===
    try:
        print(f"=== Fitting model for {state} ===")
        # Perform Bayesian parameter estimation
        timestamp, best_params, R0 = fit_function(state, train_start_date, train_end_date, progress_bar_bool=False)
        
        # Load fitted model components
        posterior_data, metadata, _ = load_posterior_results(timestamp, state)
        
        # Store training results
        fit_results_by_state[state] = {
            'timestamp': timestamp,
            'best_params': best_params,
            'R0': R0,
            'posterior_data': posterior_data,
            'metadata': metadata
        }
    except Exception as e:
        print(f"Error fitting state {state}: {e}")
    
    # === FORECASTING PHASE ===
    try:
        print(f"=== Forecasting model for {state} ===")
        # Extract fitted model components
        posterior_data = fit_results_by_state[state]['posterior_data']
        metadata = fit_results_by_state[state]['metadata']
        
        # Generate probabilistic forecasts
        forecast_df, control_cases = forcast_function(state, forecast_start_date, forecast_end_date,
                                                      metadata, posterior_data)
        
        # Store forecast results
        forecast_results_by_state[state] = {
            'forecast': forecast_df,
            'observed': control_cases
        }
    except Exception as e:
        print(f"Error forecasting state {state}: {e}")    
    
    # === COMPETITION SUBMISSION ===
    if upload is True:
        try:
            print(f"=== Uploading results for {state} ===")
            # Submit forecasts to competition platform
            upload_forecast(state, forecast_df, adm_1_map)
        except Exception as e:
            print(f"Error uploading state {state}: {e}")